In [51]:
import requests
import json
import pandas as pd    # 引用套件並縮寫為 pd
from bs4 import BeautifulSoup
# https://www.dcard.tw/f/makeup

GET 美妝版首頁，Use BeautifulSoup with html, wrtie a txt

 

In [58]:
test = open("data/test.txt","w",encoding='UTF-8')

p = requests.Session()                            # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/f/makeup") # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
a=[]
for link in sel:
    a.append(link["href"])
    print('https://www.dcard.tw'+link.get('href'))

https://www.dcard.tw/f/makeup/p/235022153
https://www.dcard.tw/f/makeup/p/235021477
https://www.dcard.tw/f/makeup/p/235025010
https://www.dcard.tw/f/makeup/p/235022346
https://www.dcard.tw/f/makeup/p/235021551
https://www.dcard.tw/f/makeup/p/235026845
https://www.dcard.tw/f/makeup/p/235025318
https://www.dcard.tw/f/makeup/p/235021107
https://www.dcard.tw/f/makeup/p/235023535
https://www.dcard.tw/f/makeup/p/235025794
https://www.dcard.tw/f/makeup/p/235026239
https://www.dcard.tw/f/makeup/p/235029187
https://www.dcard.tw/f/makeup/p/235024053
https://www.dcard.tw/f/makeup/p/235028318
https://www.dcard.tw/f/makeup/p/235029094
https://www.dcard.tw/f/makeup/p/235023273
https://www.dcard.tw/f/makeup/p/235021138
https://www.dcard.tw/f/makeup/p/235024620
https://www.dcard.tw/f/makeup/p/235027858
https://www.dcard.tw/f/makeup/p/235024257
https://www.dcard.tw/f/makeup/p/235023687
https://www.dcard.tw/f/makeup/p/235025935
https://www.dcard.tw/f/makeup/p/235024483
https://www.dcard.tw/f/makeup/p/23

In [59]:
#a
url = "https://www.dcard.tw"+ a[1]
url

'https://www.dcard.tw/f/makeup/p/235021477'

根據把首頁往下滑，發現網站會對server送一個GET請求下一個30篇文章
- 2018(G) https://ithelp.ithome.com.tw/articles/10203216
- 2019(G) https://tlyu0419.github.io/2019/04/06/Crawl-Dcard/
- 2020 https://blog.jiatool.com/posts/dcard_api_v2/

### API root

- 看板資訊 http://dcard.tw/_api/forums
- 文章資訊 http://dcard.tw/_api/forums/{看板名稱}/posts
- 文章內文	http://dcard.tw/_api/posts/{文章編號}
- 文章內引用連結	http://dcard.tw/_api/posts/{文章編號}/links
- 文章內留言	http://dcard.tw/_api/posts/{文章編號}/comments

### API wirte
- 文章數量 https://www.dcard.tw/_api/posts?limit=100
- 文章分頁 http://dcard.tw/_api/posts/?before=文章id (before 回傳由新到舊 after舊到新)
- 文章留言 http://dcard.tw/_api/posts/文章id/comments
- 留言分頁 http://dcard.tw/_api/posts/文章id/comments?after=floor數字
- 熱門文章 https://dcard.tw/_api/posts?popular=true

### others
文章資訊(meta) 與 文章內留言 預設使用熱門度 (popularity) 作為排序，而且一次請求 (request) 中只回應 30 筆。
* 想要讓這兩項使用 時間 排序，可在 GET 參數中加入 popular=false
* 欲取得更多的 文章資訊(meta)，可以使用 before={某文章編號}來獲得早於 #{某文章編號} 的另外 30 筆 資訊。
* 欲取得更多的 文章內留言，可以使用 after={某樓層} 來獲得大於 #{某樓層} 的另外 30 筆 留言。

In [64]:
def Catch(ID):
    url = 'https://www.dcard.tw/_api/posts/'+str(ID)
    requ = requests.get(url)
    rejs = requ.json()
    #rejs
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'excerpt':rejs['excerpt'],
          'createdAt':rejs['createdAt'],
          'updatedAt': rejs['updatedAt' ],
          'forumName':rejs['forumName'],
          'forumAlias':rejs['forumAlias'],
          'gender':rejs['gender'],
          'topics':rejs['topics']}],
        columns=['ID', 'title', 'content', 'excerpt', 'createdAt', 
                 'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
          )
Catch(235020702)

ID         title                                            content  \
0  235020702  發現冬天好適合用粉霜喔!  入冬後我臉已開始乾TT\n之前夏天用MUF粉餅 遮瑕力跟控油力都不錯\n但冬天用真的不夠保濕...   

                                             excerpt  \
0  入冬後我臉已開始乾TT，之前夏天用MUF粉餅 遮瑕力跟控油力都不錯，但冬天用真的不夠保濕，最...   

                  createdAt                 updatedAt forumName forumAlias  \
0  2020-12-21T02:26:52.040Z  2020-12-21T02:34:47.300Z        美妝     makeup   

  gender  topics   
0      F      NaN

In [72]:
url = 'https://www.dcard.tw/_api/posts/235020702'
requ = requests.get(url)
rejs = requ.json()
#rejs

In [68]:
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true'
requ = requests.get(url)
rejs = requ.json()
rejs

[{'id': 235022153,
  'title': '❄️聖誕妝容推薦❄️ 準備漂亮過節讓你 bling bling 惹人愛！',
  'excerpt': '這週就是聖誕節惹️，各位卡友們餐廳跟交換禮物都準備好了！，現在應該是萬事俱備 只欠東風，是聖誕妝容啦！！，來～不用擔心，小天使幫各位整理了卡友們分享的聖誕妝容，這篇真的要好好閱讀  當天亮麗登場！，這邊',
  'anonymousSchool': False,
  'anonymousDepartment': False,
  'pinned': True,
  'forumId': 'cbd5285f-3cba-4bfc-86d0-1ab52d201459',
  'replyId': None,
  'createdAt': '2020-12-21T07:07:47.084Z',
  'updatedAt': '2020-12-21T10:13:21.728Z',
  'commentCount': 10,
  'likeCount': 278,
  'withNickname': True,
  'tags': [],
  'topics': ['妝容', '分享', '聖誕妝', '美妝小天使', '精華文'],
  'meta': {'layout': 'classic', 'isModerator': True},
  'forumName': '美妝',
  'forumAlias': 'makeup',
  'nsfw': False,
  'gender': 'D',
  'school': '美妝小天使',
  'department': 'makeup_angel',
  'replyTitle': None,
  'mediaMeta': [{'id': '6b76144f-fe2b-4346-8d59-e9c4eca4ce65',
    'url': 'https://i.imgur.com/JGqeHb2l.jpg',
    'normalizedUrl': 'https://i.imgur.com/JGqeHb2l.jpg',
    'thumbnail': 'https://i.imgur.com/JGqeHb2l.jpg',
    'type': 'imag

In [44]:
# 一次讀取100篇最熱門的文章
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(Crawl(rejs[i]['id']),ignore_index=True)
print(df.shape)
df

欄位	說明	備註
ID	編號	
title	標題	
content	內容	
excerpt	摘要	
createdAt	發布時間	
updatedAt	更新時間	
commentCount	留言數	
forumName	分類	中文
forumAlias	分類	英文
gender	性別	
likeCount	心情數量	
reactions	心情細項	把以上心情細分為「愛心」、「哈哈」、「跪」、「森77」、「驚訝」等類型
topics	標籤	

In [61]:
for k in range(0,10):
    post_data={
        "before":a[-1][9:18],
        "limit":"30",
        "popular":"true"
    }
    r = p.get("https://www.dcard.tw/_api/forums/makeup/posts",
              params=post_data, 
              headers = { "Referer": "https://www.dcard.tw/", 
                         "User-Agent": "Mozilla/5.0" })
    ##
    data2 = json.loads(r.text)
    for u in range(len(data2)):
        #Temporary_url = "/f/makeup/p/"+ str(data2[u]["id"]) + "-" + str(data2[u]["title"].replace(" ","-"))
        Temporary_url = str(data2[u]["id"])
        a.append(Temporary_url)


KeyError: 0

In [56]:
for k in range(0,10):
    post_data={
        "before":a[-1][9:18],
        "limit":"30",
        "popular":"true"
    }
    r = p.get("https://www.dcard.tw/_api/forums/makeup/posts",
              params=post_data, 
              headers = { "Referer": "https://www.dcard.tw/", 
                         "User-Agent": "Mozilla/5.0" })
    ##
    data2 = json.loads(r.text)
data2

{'error': 1107, 'message': '"before" must be a number', 'field': 'before'}

In [ ]:
for k in range(0,10):
    post_data={
        "before":a[-1][9:18],
        "limit":"30",
        "popular":"true"
    }
    r = p.get("https://www.dcard.tw/_api/forums/pet/posts",
              params=post_data, 
              headers = { "Referer": "https://www.dcard.tw/", "User-Agent": "Mozilla/5.0" })
    data2 = json.loads(r.text)
    for u in range(len(data2)):
        Temporary_url = "/f/pet/p/"+ str(data2[u]["id"]) + "-" + str(data2[u]["title"].replace(" ","-"))
        a.append(Temporary_url)
j=0 #為了印頁數
q=0 #為了印張數

In [ ]:
# for k in range(0,10):
#     post_data={
#         "before":a[-1][9:18],
#         "limit":"30",
#         "popular":"true"
#     }
#     r = p.get("https://www.dcard.tw/_api/forums/pet/posts",
#               params=post_data, headers = { "Referer": "https://www.dcard.tw/", "User-Agent": "Mozilla/5.0" })

發現GET來的檔案格式是JSON，那小編比較不會處理JSON怎辦?可以把他轉成Python就好啦，並組合好網址放進list

In [ ]:
data2 = json.loads(r.text)
for u in range(len(data2)):
    Temporary_url = "/f/pet/p/"+ str(data2[u]["id"]) + "-" + str(data2[u]["title"].replace(" ","-"))
    a.append(Temporary_url)

接下來就是將list裡的URL，GET網頁出來

In [ ]:
j=0
q=0
for i in a[2:]:
    url = "https://www.dcard.tw"+i
    j+=1
    print ("第",j,"頁的URL為:"+url)
    #file.write("temperature is {} wet is {}%\n".format(temperature, humidity))
    test.write("第 {} 頁的URL為: {} \n".format(j,url))
    url=requests.get(url)
    soup = BeautifulSoup(url.text,"html.parser")

運用BeautifulSoup查看是否有符合圖片的標籤，然後運用上次所學，將圖片存檔

In [ ]:
sel_jpg = soup.select("div.Post_content_NKEl9 div div div img.GalleryImage_image_3lGzO")
for c in sel_jpg:
    q+=1
    print("第",q,"張:",c["src"])
    test.write("%\n""第 {} 張: {} \n".format(q,c["src"])) 
    pic=requests.get(c["src"])
    img2 = pic.content
    pic_out = open("spider/pet/"+str(q)+".png",'wb')
    pic_out.write(img2)
    pic_out.close()

In [ ]:
test.close()
print("爬蟲結束")

完整的程式碼(可改range將文章變多)

In [ ]:
#藉由首頁取得所有文章的URL
import requests
from bs4 import BeautifulSoup
import json

test = open("data/test.txt","w",encoding='UTF-8')


p = requests.Session()
url=requests.get("https://www.dcard.tw/f/pet")
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("div.PostList_wrapper_2BLUM a.PostEntry_root_V6g0r")
a=[]
for s in sel:
    a.append(s["href"])
url = "https://www.dcard.tw"+ a[2]

for k in range(0,10):
        post_data={
            "before":a[-1][9:18],
            "limit":"30",
            "popular":"true"
        }
        r = p.get("https://www.dcard.tw/_api/forums/pet/posts",params=post_data, headers = { "Referer": "https://www.dcard.tw/", "User-Agent": "Mozilla/5.0" })
        data2 = json.loads(r.text)
        for u in range(len(data2)):
            Temporary_url = "/f/pet/p/"+ str(data2[u]["id"]) + "-" + str(data2[u]["title"].replace(" ","-"))
            a.append(Temporary_url)
j=0 #為了印頁數
q=0 #為了印張數
for i in a[2:]:
    url = "https://www.dcard.tw"+i
    j+=1
    print ("第",j,"頁的URL為:"+url)
    #file.write("temperature is {} wet is {}%\n".format(temperature, humidity))
    test.write("第 {} 頁的URL為: {} \n".format(j,url))
    url=requests.get(url)
    soup = BeautifulSoup(url.text,"html.parser")
    sel_jpg = soup.select("div.Post_content_NKEl9 div div div img.GalleryImage_image_3lGzO")
    for c in sel_jpg:
        q+=1
        print("第",q,"張:",c["src"])
        test.write("%\n""第 {} 張: {} \n".format(q,c["src"])) 
        pic=requests.get(c["src"])
        img2 = pic.content
        pic_out = open("spider/pet/"+str(q)+".png",'wb')
        pic_out.write(img2)
        pic_out.close()

test.close()
print("爬蟲結束")